In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.qa.ablation import answer_question_standard, answer_question_cte
from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark as benchmark_single
from bellek.musique.multihop import benchmark as benchmark_multi

set_seed(89)

/home/pc/anaconda3/envs/baris/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
pd.options.display.float_format = '{:,.3f}'.format

In [4]:
from tqdm.auto import tqdm
tqdm.pandas()

In [6]:
report_df = pd.read_json("./our-method-report-20240726-001458.jsonl", orient='records', lines=True)
report_df.head()

,qdecomp,context,retrieval,top_k,qa,run,exact_match,f1
0,False,paragraphs,bm25,3,standard,1,0.190,0.287
1,False,paragraphs,semantic,3,standard,1,0.280,0.372
2,False,paragraphs,bm25,5,standard,1,0.250,0.358
3,False,paragraphs,semantic,5,standard,1,0.270,0.393
4,False,paragraphs,bm25,10,standard,1,0.260,0.386


In [10]:
manual_df = pd.DataFrame.from_records([
    {"qdecomp": True, "context": "triplets", "retrieval": "graph-search", "top_k": 10, "qa": "standard", "exact_match": 0.440, "f1": 0.519, "run": 1},
{"qdecomp": True, "context": "paragraphs+triplets", "retrieval": "graph-search", "top_k": 10, "qa": "standard", "exact_match": 0.510, "f1": 0.604, "run": 1}
])
manual_df.head()

,qdecomp,context,retrieval,top_k,qa,exact_match,f1,run
0,True,triplets,graph-search,10,standard,0.440,0.519,1
1,True,paragraphs+triplets,graph-search,10,standard,0.510,0.604,1


In [14]:
all_report_df = pd.concat([report_df, manual_df], axis=0).reset_index(drop=True)
all_report_df

,qdecomp,context,retrieval,top_k,qa,run,exact_match,f1
0,False,paragraphs,bm25,3,standard,1,0.190,0.287
1,False,paragraphs,semantic,3,standard,1,0.280,0.372
2,False,paragraphs,bm25,5,standard,1,0.250,0.358
3,False,paragraphs,semantic,5,standard,1,0.270,0.393
4,False,paragraphs,bm25,10,standard,1,0.260,0.386
...,...,...,...,...,...,...,...,...
85,True,triplets,semantic,9,standard,2,0.400,0.495
86,True,triplets,bm25,15,standard,2,0.430,0.526
87,True,triplets,semantic,15,standard,2,0.450,0.531
88,True,triplets,graph-search,10,standard,1,0.440,0.519


In [15]:
avg_report_df = all_report_df.drop(columns=['run']).groupby(['qdecomp', 'context', 'retrieval', 'top_k', 'qa']).mean().sort_values("exact_match")
avg_report_df

exact_match    f1
qdecomp context             retrieval    top_k qa                         
False   paragraphs          bm25         3     cte             0.170 0.269
                                               standard        0.190 0.293
        paragraphs+triplets semantic     5     standard        0.190 0.326
                                         3     standard        0.190 0.326
                                         10    standard        0.195 0.326
        paragraphs          bm25         5     cte             0.200 0.308
        triplets            semantic     15    standard        0.205 0.321
        paragraphs+triplets bm25         10    standard        0.220 0.366
                                         5     standard        0.220 0.371
                                         3     standard        0.220 0.368
        triplets            bm25         15    standard        0.225 0.351
                                         25    standard        0.230 0.345
                            semantic     25    standard        0.240 0.359
        paragraphs          bm25         5     standard        0.250 0.355
                                         10    standard        0.265 0.391
                            semantic     3     cte             0.270 0.378
                                         5     cte             0.275 0.373
                                               standard        0.275 0.392
                            bm25         10    cte             0.280 0.388
                            semantic     10    standard        0.285 0.419
                                         3     standard        0.290 0.380
                                         10    cte             0.310 0.403
True    triplets            bm25         9     standard        0.360 0.456
                            semantic     9     standard        0.405 0.499
                            bm25         15    standard        0.425 0.518
                            graph-search 10    standard        0.440 0.519
                            semantic     15    standard        0.450 0.529
        paragraphs          bm25         5     cte             0.490 0.591
        paragraphs+triplets bm25         3     standard        0.500 0.573
                                         10    standard        0.510 0.594
                            semantic     3     standard        0.510 0.609
                            graph-search 10    standard        0.510 0.604
                            bm25         5     standard        0.510 0.588
        paragraphs          bm25         3     cte             0.510 0.625
                            semantic     10    cte             0.520 0.623
        paragraphs+triplets semantic     10    standard        0.520 0.617
        paragraphs          semantic     5     standard        0.525 0.609
        paragraphs+triplets semantic     5     standard        0.525 0.622
        paragraphs          bm25         3     standard        0.535 0.619
                                         10    standard        0.535 0.623
                            semantic     3     cte             0.540 0.624
                                         10    standard        0.540 0.626
                                         3     standard        0.545 0.635
                            bm25         5     standard        0.545 0.628
                            semantic     5     cte             0.555 0.661
                            bm25         10    cte             0.560 0.637

In [19]:
print(avg_report_df.to_latex())

\begin{tabular}{lllllrr}
\toprule
 &  &  &  &  & exact_match & f1 \\
qdecomp & context & retrieval & top_k & qa &  &  \\
\midrule
\multirow[t]{22}{*}{False} & \multirow[t]{2}{*}{paragraphs} & \multirow[t]{2}{*}{bm25} & \multirow[t]{2}{*}{3} & cte & 0.170000 & 0.268718 \\
 &  &  &  & standard & 0.190000 & 0.292683 \\
\cline{2-7} \cline{3-7} \cline{4-7}
 & \multirow[t]{3}{*}{paragraphs+triplets} & \multirow[t]{3}{*}{semantic} & 5 & standard & 0.190000 & 0.326358 \\
\cline{4-7}
 &  &  & 3 & standard & 0.190000 & 0.326358 \\
\cline{4-7}
 &  &  & 10 & standard & 0.195000 & 0.325525 \\
\cline{2-7} \cline{3-7} \cline{4-7}
 & paragraphs & bm25 & 5 & cte & 0.200000 & 0.308480 \\
\cline{2-7} \cline{3-7} \cline{4-7}
 & triplets & semantic & 15 & standard & 0.205000 & 0.321007 \\
\cline{2-7} \cline{3-7} \cline{4-7}
 & \multirow[t]{3}{*}{paragraphs+triplets} & \multirow[t]{3}{*}{bm25} & 10 & standard & 0.220000 & 0.366213 \\
\cline{4-7}
 &  &  & 5 & standard & 0.220000 & 0.370713 \\
\cline{4-7}
 & 